In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import spatial
import freud


In [2]:
def get_order_product(df):
    tri = spatial.Delaunay(df[['x', 'y']].values)
    orders = df['hexatic_order'].values
    list_indices, point_indices = tri.vertex_neighbor_vertices
    order_product = [
        np.prod(orders[point_indices[list_indices[i]:list_indices[i+1]]])
        for i in range(len(orders))
    ]
    df['hexatic_order_product'] = order_product
    return df


In [22]:
def get_clusters(df, thresh, col):
    order = df[col].values
    above = np.abs(order) > thresh
    points = df.loc[above, ['x', 'y']].copy()
    points['z'] = 0
    points = points.values
    box = freud.box.Box(Lx=2*df.x.max(), Ly=2*df.y.max(), is2D=True)
    cl = freud.cluster.Cluster()
    cl.compute((box, points), neighbors={'r_max': 30})
    df['cluster'] = -1
    df.loc[above, 'cluster'] = cl.cluster_idx
    return df

In [4]:
def plot_clusters(df):
    plt.figure()
    df0 = df.loc[df.cluster != -1]
    df1 = df.loc[df.cluster == -1]
    plt.scatter(df0.x, df0.y, c=df0.cluster, cmap='tab20')
    plt.scatter(df1.x, df1.y, c='k')

In [5]:
def plot_order_prod(df):
    plt.figure()
    plt.scatter(df.x, df.y, c=np.abs(df.hexatic_order_product))


In [18]:
def plot_order(df):
    plt.figure()
    plt.scatter(df.x, df.y, c=np.abs(df.hexatic_order))

In [6]:
fname = "/media/data/Data/BallBearing/SmallerSpacings/2,00mm.hdf5"
data = pd.read_hdf(fname)

In [7]:
data = data.groupby('frame').apply(get_order_product)


In [8]:
data = data.groupby('frame').apply(get_clusters, 0.2)

In [9]:
max_frame = data.index.max()
%matplotlib auto
plot_clusters(data.loc[max_frame])

Using matplotlib backend: Qt5Agg


In [ ]:
plot_order_prod(data.loc[0])

In [23]:
files = [
    "/media/data/Data/BallBearing/SmallerSpacings/2,00mm.hdf5",
    "/media/data/Data/BallBearing/SmallerSpacings/1,97mm.hdf5",
    "/media/data/Data/BallBearing/SmallerSpacings/1,95mm.hdf5",
    "/media/data/Data/BallBearing/SmallerSpacings/1,93mm.hdf5",
    "/media/data/Data/BallBearing/SmallerSpacings/1,90mm.hdf5"
]
lengths = [2.0, 1.97, 1.95, 1.93, 1.90]

for file, l in zip(files, lengths):
    data = pd.read_hdf(file, 'df')
    data = data.groupby('frame').apply(get_order_product)
    data = data.groupby('frame').apply(get_clusters, 0.8, 'hexatic_order')
    data.to_hdf(file, 'df')
    max_frame = data.index.max()
    plot_clusters(data.loc[max_frame])
    plt.title(l)